#### Задача: собрать модель предсказывающую стоимость квартир из датасета с 10 000 значений, предсказать стоимость квартир в тренировочном датасете на 5000 значений и сохранить полученные предсказания для последующей сверки.  

### Осуществляем импорт необходимых библиотек

In [162]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

### Импортируем данные

In [163]:
dataset = pd.read_csv('/Users/konstantinx/opt/train.csv', delimiter=',')
dataset.head(5)

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
0,11809,27,3.0,115.027311,NaN,10.0,4,10.0,2014,0.075424,B,B,11,3097,0,NaN,0,0,B,305018.871089
1,3013,22,1.0,39.832524,23.169223,8.0,7,8.0,1966,0.118537,B,B,30,6207,1,1183.0,1,0,B,177734.553407
2,8215,1,3.0,78.342215,47.671972,10.0,2,17.0,1988,0.025609,B,B,33,5261,0,240.0,3,1,B,282078.720850
3,2352,1,1.0,40.409907,NaN,1.0,10,22.0,1977,0.007122,B,B,1,264,0,NaN,0,1,B,168106.007630
4,13866,94,2.0,64.285067,38.562517,9.0,16,16.0,1972,0.282798,B,B,33,8667,2,NaN,0,6,B,343995.102962


### Создаём функцию обогащения данных в датасете, которая убирает нулевые значения, заменяет буквенные цифрами, обрабатывает некорректные значения, например "бОльшая жилая площадь, чем вся площадь в квартире".

In [164]:
def enrichment(in_dataset):
    
    in_dataset = in_dataset.drop('Id', axis=1) 
    in_dataset['LifeSquare'] = np.where(in_dataset['LifeSquare']>in_dataset['Square'], in_dataset['Square']-in_dataset['KitchenSquare'], in_dataset['LifeSquare'])
    in_dataset['LifeSquare'] = np.where(in_dataset['LifeSquare'].isna(), in_dataset['Square']-in_dataset['KitchenSquare'], in_dataset['LifeSquare'])
    in_dataset['HouseFloor'] = np.where(in_dataset['HouseFloor']<in_dataset['Floor'], in_dataset['Floor'], in_dataset['HouseFloor'])
    in_dataset['Rooms'] = np.where(in_dataset['Rooms'] == 0, in_dataset['Rooms'].mean(), in_dataset['Rooms'])
    in_dataset['Healthcare_1'] = np.where(in_dataset['Healthcare_1'].isna(), in_dataset['Healthcare_1'].mean(), in_dataset['Healthcare_1'])
    in_dataset['Ecology_2'] = np.where(in_dataset['Ecology_2']=='A', 0, 1)
    in_dataset['Ecology_3'] = np.where(in_dataset['Ecology_3']=='A', 0, 1)
    in_dataset['Shops_2'] = np.where(in_dataset['Shops_2']=='A', 0, 1)
    in_dataset['DistrictId_SocHealth'] = in_dataset['DistrictId']+in_dataset['Helthcare_2']+in_dataset['Social_1']
    in_dataset['DistrictId_Soc1'] = in_dataset['DistrictId']+in_dataset['Social_1']
    in_dataset['DistrictId_Health'] = in_dataset['DistrictId']+in_dataset['Helthcare_2']
    in_dataset['DistrictId_Soc2'] = in_dataset['Social_2']+in_dataset['DistrictId']
    
    return(in_dataset)

### Функция отбора  наименований наиболее ценных значений датасета с помощью матрицы кореляции.

In [165]:
def corelation(dataset):
    matr = dataset.corr()
    high_corr = matr['Price']
    high_corr = high_corr[(high_corr > 0.3) | (high_corr < -0.3)]
    names = list(high_corr.index)
    names.remove('Price')
    return(names)

### Убираем выбросы с помощью межквартильного размаха (с немного изменёнными вводными).

In [166]:
def normalization(dataset):
    normal_dataset = dataset[dataset['Square'] < dataset['Square'].describe()['75%']*1.7]
    normal_dataset = normal_dataset[normal_dataset['Square']>normal_dataset['Square'].describe()['25%']*0.6]
    return(normal_dataset)

### Обрабатываем данные: 
##### 1. Удаляем выбросы
##### 2. Отбираем целевые значения
##### 3. Обогащаем данные синтетическими признаками
##### 4. Строим матрицу кореляций для проверки, будет ли модель лучше работать на меньшем количестве признаков
##### 5. Убираем целевые значения из обрабатываемого датасета
##### 6. Создаём датафрейм только с сильными признаками

In [167]:
normal_dataset = normalization(dataset) 
y = normal_dataset['Price']
X_norm = enrichment(normal_dataset)
corr_names = corelation(X_norm)
X_norm = X_norm.drop('Price', axis=1)
corr_dataset = X_norm[corr_names]


### Создаём тестовые и валидационные данные основного датасета
### Создаём модель и обучаем её на тестовых данных. 
### Создаём предсказание на валидационных данных.
### Считаем точность

In [168]:
X_train, X_valid, y_train, y_valid = train_test_split(X_norm, y, test_size=0.2, random_state=42)

model_1 = RandomForestRegressor(n_estimators=1000, max_depth=22, random_state=7)
model_1.fit(X_train, y_train)
model_pred = model_1.predict(X_valid)

r2_score(y_valid, model_pred)

0.7580677629100192

### Создаём модель на малых данных. Убедились, что уменьшение признаков ухудшает результат, поэтому берём полный вариант

In [169]:
X2_train, X2_valid, y2_train, y2_valid = train_test_split(corr_dataset, y, test_size=0.2, random_state=42)

model_2 = RandomForestRegressor(n_estimators=1000, max_depth=22, random_state=7)
model_2.fit(X2_train, y2_train)
model_pred = model_2.predict(X2_valid)

r2_score(y2_valid, model_pred)

0.6499802000109687

### Загружаем тестовые данные

In [170]:
testset = pd.read_csv('/Users/konstantinx/opt/test.csv', delimiter=',')

### Обрабатываем тестовые данные

In [171]:
norm_test = enrichment(testset)
norm_test.head(5)

,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,...,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,DistrictId_SocHealth,DistrictId_Soc1,DistrictId_Health,DistrictId_Soc2
0,44,1.0,36.847630,19.094182,5.0,5,9.0,1970,0.036122,1,...,4378,0,1036.000000,1,1,1,69,68,45,4422
1,62,1.0,42.493907,32.493907,10.0,7,17.0,2017,0.072158,1,...,629,1,1146.657263,0,0,0,64,64,62,691
2,27,2.0,59.463678,50.463678,9.0,19,19.0,1977,0.211401,1,...,1892,0,1146.657263,0,1,1,36,36,27,1919
3,23,3.0,49.646030,33.893825,6.0,2,2.0,1965,0.014073,1,...,475,0,1146.657263,0,0,1,25,25,23,498
4,74,1.0,53.837056,52.837056,1.0,8,17.0,1977,0.309479,1,...,7715,4,990.000000,0,6,1,109,109,74,7789


### Строим тестовые значения

In [172]:
model_test_pred = model_1.predict(norm_test)
model_test_pred

array([157763.30458747, 118496.27281978, 143049.91588588, ...,
       145268.98209551, 193619.24107261, 272013.98380932])

### Создаём файл с ответами

In [173]:
answer = pd.DataFrame()
answer['Id']=testset['Id']
answer['Price'] = model_test_pred
answer.to_csv('/Users/konstantinx/opt/KStarkov_predictions.csv', index=False)